In [1]:
from nltk.metrics.distance import edit_distance

In [2]:
correct_words = ['gulshan e hadeed', 'gulshan e iqbal', 'defence', 'clifton', 'meher plaza', 'al murtaza heights', 'al murtaza height']

incorrect_words= ['gulshen iqbal', 'defnse', 'klifton', 'mehar plaza', 'al murteza heights']

for word in incorrect_words:
		temp = [(edit_distance(word, w),w) for w in correct_words]
		# print(sorted(temp, key = lambda val:val[0])[0][1])

		if len(temp) > 1: 
			temp = [min(temp, key=lambda t: t[0])]

		print(temp)
		# print(temp[0][1])

# print(edit_distance(incorrect_words[2], correct_words[3]), correct_words[3])

[(3, 'gulshan e iqbal')]
[(2, 'defence')]
[(1, 'clifton')]
[(1, 'meher plaza')]
[(1, 'al murtaza heights')]


In [3]:
import json
import string

def remove_punctuation(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

def lowercase_text(text):
    return text.lower()

def process_json(json_data):
    processed_data = {}
    for key, value in json_data.items():
        processed_key = remove_punctuation(lowercase_text(key))
        processed_value = remove_punctuation(lowercase_text(value))
        processed_data[processed_key] = processed_value
    return processed_data

# Read the JSON file
filename = "abbreviations.json"
with open(filename) as file:
    data = json.load(file)

# Process the JSON data
processed_data = process_json(data)

# Write the processed data back to the JSON file
with open(filename, "w") as file:
    json.dump(processed_data, file, indent=4)

print(f"JSON file '{filename}' has been modified.")



JSON file 'abbreviations.json' has been modified.


In [ ]:
sample1 = 'House # C38, Block 8, Gulshan-e-Iqbal, Karachi'
sample2 = 'House No. 123, Street 5, Phase 7, DHA, Karachi'
sample3 = "House No. 123, St 5, Phase 7, DHA, Karachi"
sample4 = 'Apartment/Suite# 404 4th Floor, Building All Blocks, Fatima Tower, Jamaludin Afghani Rd, Block 3, PECHS, Karachi'

samples = [' House # F-37 Off, Khayaban-e-Iqbal, Block 8, Clifton, Karachi ', 
           ' House # 82-B ,1st Floor, 21st Street, Khayaban e badar, Phase 6, Defence, Karachi ', 
           ' Apartment/Suite# 702 7th Floor, Building All Block, Jheel ParkView, Siraj Road, Block 2, PECHS, Karachi ', 
           ' House # R-18, Street No 37, Block-1, Gulistan-e-Johar, Karachi ', 
           ' Apartment/Suite# 409 4th fl, Building All Blocks, Sahil Promenade, Khayaban e Sadi, Block 3, Clifton, Karachi ', 
           ' House # 6C 1st Floor Ismail Center, Alamgir Road, Bahadurabad, PECHS, Karachi ', 
           ' Apartment/Suite# 204 2nd Floor, Building C, Farhan Classic, Pro Abdul Ghafoor Rd, Block 12, Gulistan-e-Johar, Karachi ', 
           ' House # 16B/1 Main, National Highway, Phase 2, Defence, Karachi ', 
           ' House # 187/2/B Off, Shahrah e Qaideen, Block 2, PECHS, Karachi ', 
           ' House # R-11/1, Khayaban e Sadi, Phase 7, Defence, Karachi ']

sample5 = ' House # 82-B ,1st Floor, 21st Street, Khayaban e badar, Phase 6, Defence, Karachi '
sample6 = ' Apartment/Suite# 409 4th fl, Building All Blocks, Sahil Promenade, Khayaban e Sadi, Block 3, Clifton, Karachi '

# remaining_address = [' Building All Blocks', ' Sahil Promenade', ' Khayaban e Sadi', ' Block 3']
# remaining_address2 = [' Building All Blocks', ' Sahil Promenade', ' Khayaban e Sadi']
# remaining_address3 = [' Khayaban e Sadi', ' Block 3']
# remaining_address4 = [' Building All Blocks', ' Sahil Promenade']

# sample7 = 'Apartment/Suite# Flat 204, Building All Blocks, Pardesi Green Land Apartments, 8th Street, near baitul mukarrram majid, behind hakim saeed park, Bath Island, Clifton, Karachi'
# remaining_address = [' Building All Blocks', ' Pardesi Green Land Apartments', ' near baitul mukarrram majid', ' behind hakim saeed park', ' Bath Island']

# sample8 = ' House # 82-B ,1st Floor, near baitul mukarrram majid, behind hakim saeed park, 21st Street, Khayaban e badar, Phase 6, Defence, Karachi '
# remaining_address = [ ' near baitul mukarrram majid', ' behind hakim saeed park', ' Khayaban e badar', ' Phase 6']
# remaining_address = [ ' near baitul mukarrram majid', ' behind hakim saeed park', ' Khayaban e badar']

# length of df = 213874

Project: Address Normalization and Building Name Extraction

main focus of address parsing - building names and uniformaty

2 parts: 

- data parsing and storing in our DB (standart uniform way)
- checking in DB, if same address fields are there so directly storing fields in db from prior knowledge

Work flows: 
- storing main address db in file, read and load from file, run parse (processing), update & add into main address db, store back in file
- 2 major flow components for now: pre processing and how this (processing) where addressed is successfully parsed and placed into a df and saved

data pipeline shape to the project (proper flow plus maximum automation (function automatic calling inside another function))

  
- test data size should be 2% of original data size: 
    2% of 213874 = 4,277.48 = 4,275
    So a test df of size 4,275 randomly generated 


comma placement: 

check if commnas < 3 and street, house, appt, city, present in address string --> defense addreses (data2.csv)

for commna placement: identifier checks --> check if 2 keywords present at once, if 2 keywords present at once, then break according to keyword i.e. place comma or break accordingly

localized approach for comma placement;
- highlight areas with most issue for eg defence 
- check by condition if area == defence then check every field one by one 
- if more than 1 keyword; sperarte need based for eg for defence street its xxxx street/lane eg 23rd street/lane so place comma after that 

- ^ name this layer 2 check, place comma then break from there and place in correct field


After comma placement
- get number of unplaced entries in every address:

    - len of tokenized list from parse before calling probabilistic_identifiers function

    - total len of that tokenized address

    - index scores with token of address 
    
    - total average of 213874
    
    - data representation using mathplotlib of any tool




Using Machine learning for remaining normalization part where comma placement issue:

seperate normalized or processed data into 2 parts: 

- correctly normalized addresses;
    -   checks if all fields are full (option in street/road but anyone)
    -   manual additions

- incorrect normalized addresses due to comma placements;
    -   checks if appartment true but no appartment #, building name
    -   checks if house true but no house #
    -   manual additions

- train & run model to make test data correct



Tasks Remaining:
- make khi tickets addresses unique then test
- excel split from commas, then run on analyze to get before and after results
- building name spelling correction mechanism 
- run on whole data --> normalization (save as processed_khi...)
- run processed_khi on building name processor --> save 3 files, buidling corrected khi, unique building names and ticket and building name
- comma placement (alternative mechanism keyword search) --> localized approach
- manual check ups and additions.
- make report of full project --> Address Normalization & Builiding Name extraction using Python 
- update GitHub: readMe.md + files

Uniform address classification and normalizatinon for all pakistani addresses

Hierarchy: 

(if present because not all feilds in all addresses)

- Type --> INFERRED

- House # --> KEYWORD CHECK 

- Appartment # --> KEYWORD CHECK

- Building # --> check with index +-1 if type int/number present

- Building Name --> index +- >(len(tokenization list)/2)

- Street Number/Name --> KEYWORD CHECK

- Road Number/Name --> KEYWORD CHECK

- Area (eg: bath island, civil lines, khayaban) --> IF AVAILABLE IN END & Sub Area (phase, block, etc.) --> KEYWORD CHECK

- Neighbourhood  --> STANDART, [-2] in address string

- City --> STANDART, [-1] in address string


Each hierarchy step check:

- ~~pre_processing function --> grammer, punctuation, spacing, capitalization~~

- ~~tokenization~~

- ~~one by one tokenization feilds checks:~~

    - spelling check (where applicable like neighbour hoods)

    - checking if already in db (or give if there exists another one with least Levenshtein distance, if Levenshtein distance < 2 then change to above one)

    - ~~keyword check where to place and place~~

- ~~place in DB~~

- ~~when tokenization list empty, fill null or zero in remaining felids~~


algo for building name correction:

- load normalized df

- convert building names column to list

- create empty list for corrected building names

- pop 1 from building names column list & append to corrected building names list ( if path # 1) 

- iterate over building names column list:

    -   check if building name is not string: 

        -   append 'None' to corrected building names list 
    
    -   else:   

        -   corrected word corpus = corrected building names list

        -   ~~corrected word corpus = zohaib bhai corrected building names corpus~~ (alternate option but not functional as zohaib bhai corrected building names corpus only has 16% (35k/213K) addresses which are top locations)
            
        -   incorrect word = one word at a time from building names column list

        -   check number of substitutions required using using levenshtein distance mechanism already formulated

        -   if number retured is less than 3; change spelling 

        -   else; keep same spelling

- when iteration finished, replace corrected building names list with building names colum in normalized df

- save the updated building name df as: buidling corrected or processed khi.. 

- from the updated building name df, extract ticket and building name into a new df and save that df (for analysis)

- from anyone of the above save unique building names into a txt file or csv file


analyze stats algorithm: 
- load normalized dataset 
- check if type house but house number missing 
- check if type appartment but appartment number missing
- check if type appartment but building name missing 
- store all their counts (seperatly and together)
- analyze their count w.r.t total dataset size to get percentage accuracy
- print in good formart run analysis stats
- analysis function in data processor then import at the end

Report: 

Sources: rough.ipnyb, normalization.py, readME.md, chatgpt

- We have designed a unique and talior normalization schema universal for all Pakistani Addresses.
- file directory 
- rough notes
- complete walkthrough and explainantion (layman style) of the program
- if comma placement issue prevails then explain it. 